**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [75]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("/Users/baptiste_doyen/anaconda3/bin")
import skvideo.io
import cv2
import json
import random


from random import randint
from keras.models import Sequential, model_from_json, Model
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Input, Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

In [3]:
import where
from skvideo.utils import check_output
ffmpeg_exe = where.where('ffmpeg')[0]
print(ffmpeg_exe)

/Users/baptiste_doyen/anaconda3/bin/ffmpeg


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [26]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [27]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function $act$ return the next action to do : 
> - During training, the next action to do is random with probability $p(X\le \epsilon)$ (where $X$ follows a discrete uniform law in $\{0, 1, 2, 3\}$) and acted via learned policy with probability $1-p(X\le \epsilon)$
> - During testing, the next action to do is only acted via learned policy
> - The $epsilon$ enables the algorithm to have some exploration (via randomness) and optimization (via optimized policy) dimensions

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [32]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [88]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=10 # set small when debugging
n_action=4

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> Arrays :
> - ```position``` : used to define the current position of the agent and all the surroundings he can see.
> - ```board``` : used for the reward. The agent receives the reward saved in the board.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [28]:
class RandomAgent(Agent):
    def __init__(self, n_action=4):
        #super(RandomAgent, self).__init__()
        self.n_action = n_action

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action, size=1)[0]
        return action

***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [34]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
                
            # Save as a mp4
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [14]:
# change file permission 

import os
os.chmod("/Users/baptiste_doyen/anaconda3/bin/ffmpeg", 0o777)

In [15]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent(n_action=n_action)

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/10.0. Average score (0.0)
Win/lose count 14.0/22.0. Average score (-4.0)
Win/lose count 14.0/14.0. Average score (-2.6666666666666665)
Win/lose count 11.0/14.0. Average score (-2.75)
Win/lose count 8.0/19.0. Average score (-4.4)
Win/lose count 7.0/11.0. Average score (-4.333333333333333)
Win/lose count 10.0/10.0. Average score (-3.7142857142857144)
Win/lose count 10.0/6.0. Average score (-2.75)
Win/lose count 10.0/10.0. Average score (-2.4444444444444446)
Win/lose count 9.5/13.0. Average score (-2.55)
Win/lose count 12.5/11.0. Average score (-2.1818181818181817)
Win/lose count 9.5/8.0. Average score (-1.875)
Win/lose count 16.0/14.0. Average score (-1.5769230769230769)
Win/lose count 7.0/8.0. Average score (-1.5357142857142858)
Win/lose count 14.0/13.0. Average score (-1.3666666666666667)
Win/lose count 8.0/17.0. Average score (-1.84375)
Win/lose count 5.5/13.0. Average score (-2.176470588235294)
Win/lose count 8.0/15.0. Average score (-2.4444444444444446)
Win/lose 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> <br/> **(1)** We develop the expression using probability $p(.|s,a)$ :
> <br/> $Q^{\pi}(s,a) = E[\sum_{t\geq 0} \gamma^{t}r(s_t,a_t)| s_0 = s, a_0 = a] $
> <br/>$= r(s,a) + \sum_{s', a'}p(s',a'|s,a)E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t}r(s_t,a_t)|s_0 = s, a_0 = a, s_1 = s', a_1 = a']$
> <br/>$= r(s,a) + \sum_{s', a'}p(s',a'|s,a)\gamma E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t}r(s_t,a_t)|s_1 = s', a_1 = a']$ (using Markov property)
> <br/>$= r(s,a) + \sum_{s', a'}p(s',a'|s,a)\gamma E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t}r(s_t,a_t)|s_0 = s', a_0 = a']$
> <br/>$= r(s,a) + \sum_{s', a'}p(s',a'|s,a)\gamma Q^{\pi}(s',a')$
> <br/>$= r(s,a) + E_{(s',a') \sim p(.|s,a)}[\gamma Q^{\pi}(s',a')]$
> <br/>$= E_{(s',a') \sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')]$

> **(2)** We use the definition of $Q^*(s,a)$ :
> <br/>$Q^*(s,a) = max_{\pi} Q^{\pi}(s,a)$
> <br/>$= r(s,a) + \gamma max_{\pi} E_{(s',a') \sim p(.|s,a)}[Q^{\pi}(s',a')]$
> <br/>$= r(s,a) + \gamma max_{\pi} \sum_{s', a'}Q^{\pi}(s',a')p(s',a'|s,a)$
> <br/>$= r(s,a) + \gamma max_{\pi} \sum_{s', a'}Q^{\pi}(s',a')p(s',a'|s,a)$
> <br/>$= r(s,a) + \gamma max_{\pi} \sum_{s', a'}Q^{\pi}(s',a')p(s',a'|s,a)$
> <br/>$= r(s,a) + \gamma max_{\pi} \sum_{s'}\sum_{a'}Q^{\pi}(s',a')\pi(a'|s')p(s'|s,a)$
> <br/>$= r(s,a) + \gamma \sum_{s'}p(s'|s,a)\sum_{a'}max_{a'}Q^{*}(s',a')\pi^*(a'|s')$ (at each given state $s'$, the best action is taken with best policy $\pi^*$) 
> <br/>$= r(s,a) + \gamma \sum_{s'}\sum_{a'}max_{a'}Q^{*}(s',a')\pi^*(a'|s')p(s'|s,a)$
> <br/>$= r(s,a) + \gamma E_{s'\sim \pi^*(.|s,a)}[\max_{a'}Q^{*}(s',a')]$
> <br/>$= E_{s'\sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}Q^{*}(s',a')]$

> **(3)** $Q$ is parametrized by $\theta$ parameters and $Q^*$ can be seen as the solution of the fixed-point equation :  $FQ^* = Q^*$ where $F(.)$ is the functional operator $E_{s'\sim \pi^*(.|s,a)}[r(s,a) + \gamma \max_{a'}(.)(s',a')]$.
We can solve this equation by iterating the operator until enough convergence.
<br/>Then, the proposed loss, which is positive and upper bounded by $FQ_{\theta} - Q_{\theta} = Q_{\theta_{t+1}} - Q_{\theta_{t}}$, is plausible.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [84]:
class Memory(object):
    def __init__(self, max_memory=100, num_state=2):
        self.max_memory = max_memory
        self.action = np.zeros(max_memory)
        self.state = np.zeros((max_memory,5,5,num_state))
        self.n_state = np.zeros((max_memory,5,5,num_state))
        self.reward = np.zeros(max_memory)
        self.game_over = np.zeros(max_memory)
        self.count = 0
        self.full = False

    def remember(self, m):
        self.count += 1
        self.state[self.cursor] = m[0]
        self.n_state[self.cursor] = m[1]
        self.action[self.cursor] = m[2]
        self.reward[self.cursor] = m[3]
        self.game_over[self.cursor] = m[4]
        if self.count > self.max_memory-1:
            self.full = True
            self.count = 0

    def random_access(self):
        if self.full: 
            i = randint(0, self.max_memory-1)
        else :
            i = randint(0, self.count)
        return [self.state[i,:,:,:],  self.n_state[i,:,:,:], 
                 self.action[i], self.reward[i], self.game_over[i]]

***
The pipeline we will use for training is given below:

In [85]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [86]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2, n_action=4):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state
        
        # number of actions
        self.n_action = n_action

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        # epsilon
        self.epsilon = epsilon

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None,:,:,:]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            [s, n_s, a, r, game_over] = self.memory.random_access()
            a = int(a)
            input_states[i] = s
            target_q[i,:] = self.model.predict(s[None,:,:,:])
            if game_over_:
                target_q[i,a] = r
            else:
                target_q[i,a] = r + self.discount*np.max(self.model.predict(n_s[None,:,:,:]))

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -1, 1)

        l = self.model.train_on_batch(input_states, target_q)
        
        return l
    
            
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # Fully-Connected Neural Network Model
        
        # Input dimensioning
        input_tensor = Input(shape=(5,5,self.n_state))
        l_1 = Flatten()(input_tensor)
        
        # FC layers
        l_2 = Dense(32, activation='relu')(l_1)
        l_3 = Dense(32, activation='relu')(l_2)
        l_4 = Dense(32, activation='relu')(l_3)
        
        # Final layer
        pred = Dense(4, activation='linear')(l_4)
        
        # Model
        model = Model(inputs=input_tensor, outputs=pred)
        model.summary()
        
        # Optimization
        model.compile(Adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [87]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 5, 5, 2)           0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 32)                1632      
_________________________________________________________________
dense_50 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_51 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_52 (Dense)             (None, 4)                 132       
Total params: 3,876.0
Trainable params: 3,876.0
Non-trainable params: 0.0
_________________________________________________________________
Ep

Epoch 120/200 | Loss 0.2000 | Win/lose count 9.0/2.0 (7.0)
Epoch 121/200 | Loss 0.2143 | Win/lose count 6.0/3.0 (3.0)
Epoch 122/200 | Loss 0.2081 | Win/lose count 14.5/5.0 (9.5)
Epoch 123/200 | Loss 0.1960 | Win/lose count 9.0/5.0 (4.0)
Epoch 124/200 | Loss 0.1982 | Win/lose count 12.5/4.0 (8.5)
Epoch 125/200 | Loss 0.1988 | Win/lose count 6.5/3.0 (3.5)
Epoch 126/200 | Loss 0.2023 | Win/lose count 11.0/5.0 (6.0)
Epoch 127/200 | Loss 0.2124 | Win/lose count 2.5/2.0 (0.5)
Epoch 128/200 | Loss 0.2079 | Win/lose count 5.5/4.0 (1.5)
Epoch 129/200 | Loss 0.2167 | Win/lose count 4.5/5.0 (-0.5)
Epoch 130/200 | Loss 0.1878 | Win/lose count 3.5/4.0 (-0.5)
Epoch 131/200 | Loss 0.1996 | Win/lose count 8.5/1.0 (7.5)
Epoch 132/200 | Loss 0.2064 | Win/lose count 10.0/5.0 (5.0)
Epoch 133/200 | Loss 0.2139 | Win/lose count 9.5/6.0 (3.5)
Epoch 134/200 | Loss 0.2050 | Win/lose count 5.0/2.0 (3.0)
Epoch 135/200 | Loss 0.2030 | Win/lose count 6.0/3.0 (3.0)
Epoch 136/200 | Loss 0.2007 | Win/lose count 10.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [89]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # CNN training model
        
        # Input dimensioning
        input_tensor = Input(shape=(5, 5, self.n_state))
        
        # Convolutional layers
        conv_1 = Conv2D(16, (3, 3), strides=(2, 2), activation='relu')(input_tensor)
        conv_2 = Conv2D(32, (2, 2), strides=(1, 1), activation='relu')(conv_1)
        f_1 = Flatten()(conv_2)
        
        # Fully-Connecte Layer
        l_1 = Dense(32, activation='relu')(f_1)
        pred = Dense(4, activation='linear')(l_1)
        
        # Model
        model = Model(inputs=input_tensor, outputs=pred)
        model.summary()
        
        # Optimization 
        model.compile(Adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [90]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 5, 5, 2)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 16)          304       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 32)          2080      
_________________________________________________________________
flatten_14 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_54 (Dense)             (None, 4)                 132       
Total params: 3,572.0
Trainable params: 3,572.0
Non-trainable params: 0.0
_________________________________________________________________
Ep

Epoch 119/200 | Loss 0.2026 | Win/lose count 2.0/0 (2.0)
Epoch 120/200 | Loss 0.1994 | Win/lose count 9.5/1.0 (8.5)
Epoch 121/200 | Loss 0.1976 | Win/lose count 8.5/4.0 (4.5)
Epoch 122/200 | Loss 0.1969 | Win/lose count 4.0/5.0 (-1.0)
Epoch 123/200 | Loss 0.1996 | Win/lose count 8.5/4.0 (4.5)
Epoch 124/200 | Loss 0.2085 | Win/lose count 16.0/5.0 (11.0)
Epoch 125/200 | Loss 0.1916 | Win/lose count 9.0/0 (9.0)
Epoch 126/200 | Loss 0.1985 | Win/lose count 5.5/5.0 (0.5)
Epoch 127/200 | Loss 0.2011 | Win/lose count 13.0/1.0 (12.0)
Epoch 128/200 | Loss 0.2014 | Win/lose count 8.5/5.0 (3.5)
Epoch 129/200 | Loss 0.2103 | Win/lose count 4.5/2.0 (2.5)
Epoch 130/200 | Loss 0.2079 | Win/lose count 5.5/2.0 (3.5)
Epoch 131/200 | Loss 0.2080 | Win/lose count 7.0/1.0 (6.0)
Epoch 132/200 | Loss 0.2132 | Win/lose count 16.0/3.0 (13.0)
Epoch 133/200 | Loss 0.2186 | Win/lose count 8.5/2.0 (6.5)
Epoch 134/200 | Loss 0.2072 | Win/lose count 4.5/2.0 (2.5)
Epoch 135/200 | Loss 0.1979 | Win/lose count 7.0/4.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [96]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)

# Agent CNN
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

# Agent FC
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

# Results of the test
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 5, 5, 2)           0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 2, 16)          304       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 1, 1, 32)          2080      
_________________________________________________________________
flatten_21 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_73 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_74 (Dense)             (None, 4)                 132       
Total params: 3,572.0
Trainable params: 3,572.0
Non-trainable params: 0.0
_________________________________________________________________
__

In [ ]:
# Training and Testing with low temperature 

# Training

env = Environment(grid_size=size, max_time=T, temperature=0.15)
agent = DQN_FC(size, lr=0.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train_low_temp')

env = Environment(grid_size=size, max_time=T, temperature=0.15)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train_low_temp')

# Testing

agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

In [ ]:
# Training and Testing with high temperature 

# Training

env = Environment(grid_size=size, max_time=T, temperature=0.8)
agent = DQN_FC(size, lr=0.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train_low_temp')

env = Environment(grid_size=size, max_time=T, temperature=0.8)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train_low_temp')

# Testing

agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

> Comparison of the 2 network architecture :

> - **Differences** : 
> <br/> - CNN network tends to perform better in general compared to FC network
> <br/> - Agent trained with CNN network tends to explore a bit more the map 
> - **Similarities** : 
> <br/> - Both networks architecture are limited by repeated oscillations of the agent between 2 neighbour positions on the map 
> <br/> - With low temperatures, the agent is less performant with the 2 networks

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [ ]:
def train_explore(agent,env,epoch,prefix=''):
    pass
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        pass
    
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [ ]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***